# Azure AI-agenter med understøttelse af Model Context Protocol (MCP)

Denne notebook demonstrerer, hvordan man bruger Azure AI-agenter med Model Context Protocol (MCP)-værktøjer til at skabe en intelligent agent, der kan udnytte eksterne MCP-servere for forbedrede funktioner.


## Installer nødvendige NuGet-pakker

Først skal vi installere Azure AI Agents Persistent-pakken, som leverer den grundlæggende funktionalitet til at arbejde med Azure AI Agents.


## Fordele ved nøglefri godkendelse

Denne notebook demonstrerer **nøglefri godkendelse**, som giver flere fordele:
- ✅ **Ingen API-nøgler at administrere** - Bruger Azure-identitetsbaseret godkendelse
- ✅ **Forbedret sikkerhed** - Ingen hemmeligheder gemt i kode eller konfiguration
- ✅ **Automatisk rotation af legitimationsoplysninger** - Azure håndterer legitimationslivscyklussen
- ✅ **Rollebaseret adgang** - Bruger Azure RBAC til detaljerede tilladelser

`DefaultAzureCredential` vil automatisk bruge den bedst tilgængelige kilde til legitimationsoplysninger:
1. Managed Identity (når det kører i Azure)
2. Azure CLI-legitimationsoplysninger (under udvikling)
3. Visual Studio-legitimationsoplysninger
4. Miljøvariabler (hvis konfigureret)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installer Azure Identity-pakken til godkendelse med Azure-tjenester ved hjælp af DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importér Nødvendige Navneområder

Importér de nødvendige navneområder for Azure AI Agents og Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurer Azure AI Agent Client (Nøglefri Godkendelse)

Opsæt konfigurationsvariablerne og opret PersistentAgentsClient ved hjælp af **nøglefri godkendelse**:
- **projectEndpoint**: Azure AI Foundry-projektets endpoint
- **modelDeploymentName**: Navnet på den implementerede AI-model (GPT-4.1 nano)
- **mcpServerUrl**: URL'en til MCP-serveren (Microsoft Learn API)
- **mcpServerLabel**: En label til at identificere MCP-serveren
- **DefaultAzureCredential**: Bruger administreret identitet, Azure CLI eller andre legitimationskilder (ingen API-nøgler påkrævet)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Opret MCP-værktøjsdefinition

Opret en MCP-værktøjsdefinition, der forbinder til Microsoft Learn MCP-serveren. Dette vil give agenten adgang til Microsoft Learn-indhold og dokumentation.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Opret AI-agenten

Opret en vedvarende AI-agent med den angivne model og MCP-værktøjer. Agenten er konfigureret med:
- GPT-4.1 nano-modellen  
- Instruktioner til at bruge MCP-værktøjer til assistance  
- Adgang til Microsoft Learn MCP-serveren  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Opret tråd og send besked

Opret en samtaletråd og send en brugerbesked, der spørger om forskellen mellem Azure OpenAI og OpenAI. Dette vil teste agentens evne til at bruge MCP-værktøjerne til at give præcise oplysninger.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurer MCP-værktøjsressourcer (Nøglefri)

Opsæt MCP-værktøjsressourcerne. For en fuldstændig nøglefri tilgang kan du fjerne brugerdefinerede headers, hvis MCP-serveren understøtter Azure-identitetsbaseret godkendelse. Eksemplet nedenfor viser, hvordan man tilføjer headers, hvis det er nødvendigt, men for nøglefri scenarier er disse muligvis ikke påkrævet.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Start Agent Run

Opret og start en kørsel for at behandle brugerens besked. Agenten vil bruge de konfigurerede MCP-værktøjer og ressourcer til at generere et svar.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Overvåg kørsel og håndter værktøjsgodkendelser (Keyless)

Overvåg agentens kørselsstatus og håndter eventuelle nødvendige værktøjsgodkendelser. Denne proces:
1. Venter på, at kørslen afsluttes eller kræver handling
2. Godkender automatisk MCP-værktøjsanmodninger, når det er nødvendigt
3. Ved nøglefri autentifikation er headers muligvis ikke nødvendige, hvis MCP-serveren understøtter Azure-identitet


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Vise Samtaleresultater

Hent og vis alle beskeder i tråden, der viser både brugerens spørgsmål og agentens svar. Beskederne vises i kronologisk rækkefølge med tidsstempler og rolleindikatorer.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
